In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
df = pd.read_csv('data/kc_house_data.csv')

In [2]:
df['sqft_basment_calc'] = df['sqft_living'] - df['sqft_above']

In [3]:
df = df.drop('view', axis=1)

In [4]:
df["waterfront"] = df["waterfront"].fillna("Unknown")

In [5]:
df["yr_renovated"] = df["yr_renovated"].fillna(1)

In [6]:
grades = df[['grade']]
categories = [['3 Poor', '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion']]
ords = OrdinalEncoder(categories=categories)
ords.fit(grades)
ords.transform(grades)

array([[4.],
       [4.],
       [3.],
       ...,
       [4.],
       [5.],
       [4.]])

In [7]:
grades_encoded = pd.DataFrame(
 
    ords.transform(grades),
    
    index=df.index
)

In [9]:
df.drop('grade', axis=1, inplace=True)

In [15]:
df = pd.concat([df, grades_encoded], axis=1)

In [21]:
df['grades'] = df[0]

In [23]:
df.drop(0, axis=1, inplace=True)

In [25]:
waterfront_e = df[['waterfront']]
categories1 = [['NO', 'Unknown', 'YES']]
ords1 = OrdinalEncoder(categories=categories1)
ords1.fit(waterfront_e)
ords1.transform(waterfront_e)

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

In [26]:
waterfront_encoded = pd.DataFrame(
   
    ords1.transform(waterfront_e),

    index=df.index
)

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
21592,0.0
21593,0.0
21594,0.0
21595,1.0


In [27]:
df.drop('waterfront', axis=1, inplace=True)
df = pd.concat([df, waterfront_encoded], axis=1)

In [28]:
df['waterfront'] = df[0]
df.drop(0, axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 21597 non-null  int64  
 1   date               21597 non-null  object 
 2   price              21597 non-null  float64
 3   bedrooms           21597 non-null  int64  
 4   bathrooms          21597 non-null  float64
 5   sqft_living        21597 non-null  int64  
 6   sqft_lot           21597 non-null  int64  
 7   floors             21597 non-null  float64
 8   condition          21597 non-null  object 
 9   sqft_above         21597 non-null  int64  
 10  sqft_basement      21597 non-null  object 
 11  yr_built           21597 non-null  int64  
 12  yr_renovated       21597 non-null  float64
 13  zipcode            21597 non-null  int64  
 14  lat                21597 non-null  float64
 15  long               21597 non-null  float64
 16  sqft_living15      215

In [29]:
conditions_e = df[['condition']]
categories2 = [['Poor', 'Fair', 'Average', 'Good', 'Very Good']]
ords2 = OrdinalEncoder(categories=categories2)
ords2.fit(conditions_e)
ords2.transform(conditions_e)

array([[2.],
       [2.],
       [2.],
       ...,
       [2.],
       [2.],
       [2.]])

In [30]:
conditions_encoded = pd.DataFrame(
    
    ords2.transform(conditions_e),

    index=df.index
)


In [32]:
df.drop('condition', axis=1, inplace=True)
df = pd.concat([df, conditions_encoded], axis=1)

In [33]:
df['condition'] = df[0]
df.drop(0, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 21597 non-null  int64  
 1   date               21597 non-null  object 
 2   price              21597 non-null  float64
 3   bedrooms           21597 non-null  int64  
 4   bathrooms          21597 non-null  float64
 5   sqft_living        21597 non-null  int64  
 6   sqft_lot           21597 non-null  int64  
 7   floors             21597 non-null  float64
 8   sqft_above         21597 non-null  int64  
 9   sqft_basement      21597 non-null  object 
 10  yr_built           21597 non-null  int64  
 11  yr_renovated       21597 non-null  float64
 12  zipcode            21597 non-null  int64  
 13  lat                21597 non-null  float64
 14  long               21597 non-null  float64
 15  sqft_living15      21597 non-null  int64  
 16  sqft_lot15         215

In [36]:
df.drop('date', axis=1, inplace=True)
df.drop('sqft_basement', axis=1, inplace=True)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 21597 non-null  int64  
 1   price              21597 non-null  float64
 2   bedrooms           21597 non-null  int64  
 3   bathrooms          21597 non-null  float64
 4   sqft_living        21597 non-null  int64  
 5   sqft_lot           21597 non-null  int64  
 6   floors             21597 non-null  float64
 7   sqft_above         21597 non-null  int64  
 8   yr_built           21597 non-null  int64  
 9   yr_renovated       21597 non-null  float64
 10  zipcode            21597 non-null  int64  
 11  lat                21597 non-null  float64
 12  long               21597 non-null  float64
 13  sqft_living15      21597 non-null  int64  
 14  sqft_lot15         21597 non-null  int64  
 15  sqft_basment_calc  21597 non-null  int64  
 16  grades             215

In [40]:
df.to_csv('cleaned_kc_house_data.csv', index=False)